In [9]:
!pip install python-dotenv
!pip install selenium
!pip install pandas
!pip install reportlab

In [10]:
# 基本ライブラリ
import time
import math
import os

# selenium
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert

# pandas
import pandas

# reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont


In [11]:
# .env
%load_ext dotenv
%dotenv

#PATH_CHROMEDRIVER =("C:\\App\\ChromeDriver\\current\\chromedriver.exe")
PATH_CHROMEDRIVER = os.environ.get('PATH_CHROMEDRIVER')
URL_CODE39_MAKER = "https://barcode-place.azurewebsites.net/Barcode/code39"
path_download = os.getcwd() + os.path.sep + "work"
path_download

path_font = os.getcwd() + os.path.sep + "font"
FONT_BASIC = 'GenShinGothic'
registerFont(TTFont(FONT_BASIC, path_font + os.path.sep + 'GenShinGothic-Monospace-Medium.ttf'))
PDF_OUTPUT_FILENAME = 'Output.pdf'
PDF_A4_VERTICAL_W = 210 * mm
PDF_A4_VERTICAL_H = 297 * mm
# 画像実サイズ w 416px h 58px (208:29)
PDF_BARCODE_W = 75 * mm
PDF_BARCODE_H = PDF_BARCODE_W/208*29 # 10.5mm


In [18]:
df_input = pandas.read_excel('MakeBarCode.xlsx', sheet_name='DATA',
            converters={
                0:str, # 品番
                1:str, # サイズ
                2:str, # カラー
                3:str, # 上代
                4:str, # SKU
                5:str, # バーコード上段
                6:str  # バーコード下段
            })
df_input

,品番,サイズ,カラー,上代,SKU,バーコード上段,バーコード下段
0,A2201FA 053,9,02,"45,000",A2201FA 053-9 -02,A2201FA 053,9 020045000
1,A2201FA 053,9,01,"45,000",A2201FA 053-9 -01,A2201FA 053,9 010045000
2,A2201FA 053,9,03,"45,000",A2201FA 053-9 -03,A2201FA 053,9 030045000
3,A2201FA 100,3,01,"48,000",A2201FA 100-3 -01,A2201FA 100,3 010048000
4,A2201FA 100,3,02,"48,000",A2201FA 100-3 -02,A2201FA 100,3 020048000
5,A2201FA 100,4,01,"48,000",A2201FA 100-4 -01,A2201FA 100,4 010048000
6,A2201FA 100,4,02,"48,000",A2201FA 100-4 -02,A2201FA 100,4 020048000
7,A2201FA 103,9,01,"52,000",A2201FA 103-9 -01,A2201FA 103,9 010052000
8,A2201FA 103,9,03,"52,000",A2201FA 103-9 -03,A2201FA 103,9 030052000
9,A2201FA 204,3,01,"52,000",A2201FA 204-3 -01,A2201FA 204,3 010052000


In [19]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    'download.default_directory' : path_download,
    "download.prompt_for_download": False
})

driver = webdriver.Chrome(executable_path=PATH_CHROMEDRIVER, options = options)
driver.get(URL_CODE39_MAKER)
time.sleep(2)

In [20]:
pdf = canvas.Canvas(PDF_OUTPUT_FILENAME)
pdf.saveState()
pdf.setFont(FONT_BASIC, 18)
pdf.setPageSize((PDF_A4_VERTICAL_W, PDF_A4_VERTICAL_H))

rowIndex = 0
colIndex = 0
for row in df_input.itertuples():
    val_sku = row.SKU
    val_price = row.上代
    val_code39_01 = row.バーコード上段
    val_code39_02 = row.バーコード下段
    val_code39_01_image_filename = 'code39_'+ val_code39_01.replace(' ', '+') +'.png'
    #val_code39_01_image_filename = 'code39_'+ val_code39_01.replace(' ', '+') +'.eps'
    val_code39_02_image_filename = 'code39_'+ val_code39_02.replace(' ', '+') +'.png'
    #val_code39_02_image_filename = 'code39_'+ val_code39_02.replace(' ', '+') +'.eps'
    val_code39_01_image_filename_path = path_download + os.path.sep + val_code39_01_image_filename
    val_code39_02_image_filename_path = path_download + os.path.sep + val_code39_02_image_filename

    if not os.path.exists(val_code39_01_image_filename_path) or \
       not os.path.exists(val_code39_02_image_filename_path):
        driver.find_element_by_id('MainContent_TextBox1').send_keys(val_code39_01)
        driver.find_element_by_id('MainContent_TextBox2').send_keys(val_code39_02)

        driver.find_element_by_id('MainContent_LinkButtonGene').click()
        time.sleep(2)

        if not os.path.exists(val_code39_01_image_filename_path):
            driver.find_element_by_id('MainContent_LinkButtonPng1').click()
            #driver.find_element_by_id('MainContent_LinkButtonEps1').click()
        
        if not os.path.exists(val_code39_02_image_filename_path):
            driver.find_element_by_id('MainContent_LinkButtonPng2').click()
            #driver.find_element_by_id('MainContent_LinkButtonEps1').click()
            
        time.sleep(2)

        driver.find_element_by_id('MainContent_LinkButtonClear').click()
    # ----------
    
    pos_x = 50 + colIndex*270
    pos_y = PDF_A4_VERTICAL_H-(50 + rowIndex*150)
    pdf.rect(pos_x-10, pos_y+25, 230, -135)
    pdf.drawString(pos_x, pos_y, val_sku)
    pos_y -= 20
    pdf.drawString(pos_x, pos_y, val_price + ' 円')
    pos_y -= 45
    pdf.drawInlineImage(val_code39_01_image_filename_path, pos_x, pos_y, PDF_BARCODE_W, PDF_BARCODE_H)
    pos_y -= 12 * mm
    pdf.drawInlineImage(val_code39_02_image_filename_path, pos_x, pos_y, PDF_BARCODE_W, PDF_BARCODE_H)
    
    if rowIndex < 4:
        if colIndex == 0:
            colIndex = 1
        else:
            rowIndex += 1
            colIndex = 0
    else:
        if colIndex == 0:
            colIndex = 1
        else:
            pdf.showPage() # 改ページ
            pdf.setFont(FONT_BASIC, 18)
            rowIndex = 0
            colIndex = 0
    
# ----------

# PDFファイル作成
pdf.save()